In [127]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [374]:
baseurl = 'http://elections.interieur.gouv.fr/presidentielle-2017/'

In [398]:
response = requests.get(baseurl)

In [399]:
doc = BeautifulSoup(response.text, 'html.parser')

In [400]:
links = doc.select('#carte_france area')
len(links)

358

In [401]:
url_list = []
for link in links:
    href = link.get('href')
    if href:
        url_list.append((href, link.get('title')))

In [402]:
url_list = list(set(url_list))

In [403]:
len(url_list)

107

In [404]:
targetCols = ['id', 'title', 'Abstention',
 'Nom',
 'Prenom',
 'Score',
 'Nom_1',
 'Prenom_1',
 'Score_1',
 'Statut']

usecols = ['id', 'title', 'Abstention',
 'Candidat1', 'Voix', '% Inscrits', '% Exprimés',
 'Candidat2', 'Voix2', '% Inscrits2', '% Exprimés2', 'Statut']

In [405]:
url

('044/054/index.html', 'Meurthe-et-Moselle (54) - résultats complets')

In [406]:
df = pd.DataFrame(columns=usecols)

for url in url_list:
    u = url[0]
    print(url[1], '…')
    absoluteURL = baseurl + u.replace('./', '')
    response = requests.get(absoluteURL)
    docDpt = BeautifulSoup(response.text, 'html.parser')
    dpt_nom = re.match('(.*) \(', url[1]).group(1)
#    dpt_nom_check = docDpt.select('.pub-fil-ariane a')[2].string
    partiel = re.match('.* - (.*)', url[1]).group(1)

    print(dpt_nom, 'vs.', dpt_nom_check, url)
    
    no_match = re.match('.*\((.*?)\)', url[1])
    dpt_no = no_match.group(1)
    
    result_tables = docDpt.select('.tableau-resultats-listes-ER')
    if len(result_tables) > 1:
        rows = result_tables[0].select('tr')
        result_tables = docDpt.select('.tableau-resultats-listes-ER')
        rows = result_tables[0].select('tr')
        print(len(rows), 'rows found in', dpt_nom, ' – ', dpt_no)
        cleanRows = [dpt_no, dpt_nom]

        # 1) abstention
        abstention_tables = docDpt.select('.tableau-mentions')
        td_list = abstention_tables[0].select('td')
        for td in td_list:
            if td.string == 'Abstentions':
                #print(td, td.find_next_siblings()[1].string)
                abstentions = td.find_next_siblings()[1].string.strip()
                cleanRows.append(abstentions)

        # 2) candidats
        for row in rows[1:]:
            fields = row.select('td')
            fields_str = [f.string.strip() for f in fields]
            cleanRows.extend(fields_str)

        cleanRows.append(partiel)
        df = df.append(pd.DataFrame([(cleanRows)], columns=usecols))

Indre-et-Loire (37) - résultats complets …
Indre-et-Loire vs. Val-d'Oise (95) ('024/037/index.html', 'Indre-et-Loire (37) - résultats complets')
3 rows found in Indre-et-Loire  –  37
Aude (11) - résultats complets …
Aude vs. Val-d'Oise (95) ('076/011/index.html', 'Aude (11) - résultats complets')
3 rows found in Aude  –  11
Jura (39) - résultats complets …
Jura vs. Val-d'Oise (95) ('027/039/index.html', 'Jura (39) - résultats complets')
3 rows found in Jura  –  39
Yonne (89) - résultats complets …
Yonne vs. Val-d'Oise (95) ('027/089/index.html', 'Yonne (89) - résultats complets')
3 rows found in Yonne  –  89
Manche (50) - résultats complets …
Manche vs. Val-d'Oise (95) ('028/050/index.html', 'Manche (50) - résultats complets')
3 rows found in Manche  –  50
Loire (42) - résultats complets …
Loire vs. Val-d'Oise (95) ('084/042/index.html', 'Loire (42) - résultats complets')
3 rows found in Loire  –  42
Bas-Rhin (67) - résultats complets …
Bas-Rhin vs. Val-d'Oise (95) ('044/067/index.html

### NB: 79 dpts -> 86 dpts à 23h30 -> 96 à 23h56

In [407]:
df.shape

(96, 12)

In [408]:
def getNom(value):
    if value.find('MACRON') > 0:
        return 'Macron'
    elif value.find('PEN') > 0:
        return 'Le Pen'
    else:
        return False

def getPrenom(value):
    if value.find('MACRON') > 0:
        return 'Emmanuel'
    elif value.find('PEN') > 0:
        return 'Marine'

In [409]:
def cleanNumbers(value):
    return value.replace(' ', '').replace(',', '.')

In [410]:
df_backup = df.copy()

In [411]:
df.head()

,id,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Statut
0,37,Indre-et-Loire,"22,97",M. Emmanuel MACRON,201 199,"46,92","69,23",Mme Marine LE PEN,89 435,"20,86","30,77",résultats complets
0,11,Aude,"22,67",M. Emmanuel MACRON,100 897,"37,01","55,33",Mme Marine LE PEN,81 443,"29,87","44,67",résultats complets
0,39,Jura,"21,58",M. Emmanuel MACRON,79 268,"41,65","61,37",Mme Marine LE PEN,49 887,"26,21","38,63",résultats complets
0,89,Yonne,"23,56",M. Emmanuel MACRON,89 529,"37,01","55,04",Mme Marine LE PEN,73 136,"30,24","44,96",résultats complets
0,50,Manche,"21,85",M. Emmanuel MACRON,176 692,"46,62","67,23",Mme Marine LE PEN,86 133,"22,72","32,77",résultats complets


# 79 -> 86 dpts à 23h30

In [412]:
df['Nom'] = df['Candidat1'].apply(getNom)
df['Prenom'] = df['Candidat1'].apply(getPrenom)

In [413]:
df['Nom_1'] = df['Candidat2'].apply(getNom)
df['Prenom_1'] = df['Candidat2'].apply(getPrenom)

In [414]:
for col in ['Abstention', 'Voix', '% Inscrits',
       '% Exprimés', 'Voix2', '% Inscrits2', '% Exprimés2']:
    df[col] = df[col].apply(cleanNumbers)

In [415]:
df['Score'] = df['% Exprimés']
df['Score_1']  = df['% Exprimés2']

In [416]:
df['Candidat1'].apply(getNom).value_counts()

Macron    94
Le Pen     2
Name: Candidat1, dtype: int64

In [417]:
def toInt(value):
    if value not in ('2A', '2B'):
        return int(value)
    else:
        return value
df['id'] = df['id'].apply(toInt)

In [418]:
df_noms = pd.read_csv('/Users/paul/Sites/elections_france/d3-composite-projections/data-t1.csv', usecols=['id', 'title'])

In [419]:
df_noms['id'] = df_noms['id'].apply(toInt)

In [420]:
df_noms[df_noms['title'].str.contains('Bourgogne')]

,id,title


In [421]:
dfm = df.merge(df_noms, how='outer', on='title')
del dfm['id_x']
dfm['id'] = dfm['id_y']

In [422]:
dfm

,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Statut,Nom,Prenom,Nom_1,Prenom_1,Score,Score_1,id_y,id
0,Indre-et-Loire,22.97,M. Emmanuel MACRON,201199,46.92,69.23,Mme Marine LE PEN,89435,20.86,30.77,résultats complets,Macron,Emmanuel,Le Pen,Marine,69.23,30.77,37,37
1,Aude,22.67,M. Emmanuel MACRON,100897,37.01,55.33,Mme Marine LE PEN,81443,29.87,44.67,résultats complets,Macron,Emmanuel,Le Pen,Marine,55.33,44.67,11,11
2,Jura,21.58,M. Emmanuel MACRON,79268,41.65,61.37,Mme Marine LE PEN,49887,26.21,38.63,résultats complets,Macron,Emmanuel,Le Pen,Marine,61.37,38.63,39,39
3,Yonne,23.56,M. Emmanuel MACRON,89529,37.01,55.04,Mme Marine LE PEN,73136,30.24,44.96,résultats complets,Macron,Emmanuel,Le Pen,Marine,55.04,44.96,89,89
4,Manche,21.85,M. Emmanuel MACRON,176692,46.62,67.23,Mme Marine LE PEN,86133,22.72,32.77,résultats complets,Macron,Emmanuel,Le Pen,Marine,67.23,32.77,50,50
5,Loire,24.14,M. Emmanuel MACRON,218602,42.79,63.86,Mme Marine LE PEN,123713,24.21,36.14,résultats complets,Macron,Emmanuel,Le Pen,Marine,63.86,36.14,42,42
6,Haute-Loire,21.25,M. Emmanuel MACRON,76233,43.04,63.35,Mme Marine LE PEN,44110,24.90,36.65,résultats complets,Macron,Emmanuel,Le Pen,Marine,63.35,36.65,43,43
7,La Réunion,37.09,M. Emmanuel MACRON,212097,33.17,60.26,Mme Marine LE PEN,139873,21.88,39.74,résultats complets,Macron,Emmanuel,Le Pen,Marine,60.26,39.74,NaN,NaN
8,Loire-Atlantique,21.62,M. Emmanuel MACRON,530409,53.39,77.14,Mme Marine LE PEN,157184,15.82,22.86,résultats complets,Macron,Emmanuel,Le Pen,Marine,77.14,22.86,44,44
9,Ille-et-Vilaine,20.40,M. Emmanuel MACRON,403293,55.24,77.67,Mme Marine LE PEN,115942,15.88,22.33,résultats complets,Macron,Emmanuel,Le Pen,Marine,77.67,22.33,35,35


In [423]:
dfm[targetCols].to_csv('/Users/paul/Sites/elections_france/d3-composite-projections/data-t2.csv', index=False)

In [424]:
df

,id,title,Abstention,Candidat1,Voix,% Inscrits,% Exprimés,Candidat2,Voix2,% Inscrits2,% Exprimés2,Statut,Nom,Prenom,Nom_1,Prenom_1,Score,Score_1
0,37,Indre-et-Loire,22.97,M. Emmanuel MACRON,201199,46.92,69.23,Mme Marine LE PEN,89435,20.86,30.77,résultats complets,Macron,Emmanuel,Le Pen,Marine,69.23,30.77
0,11,Aude,22.67,M. Emmanuel MACRON,100897,37.01,55.33,Mme Marine LE PEN,81443,29.87,44.67,résultats complets,Macron,Emmanuel,Le Pen,Marine,55.33,44.67
0,39,Jura,21.58,M. Emmanuel MACRON,79268,41.65,61.37,Mme Marine LE PEN,49887,26.21,38.63,résultats complets,Macron,Emmanuel,Le Pen,Marine,61.37,38.63
0,89,Yonne,23.56,M. Emmanuel MACRON,89529,37.01,55.04,Mme Marine LE PEN,73136,30.24,44.96,résultats complets,Macron,Emmanuel,Le Pen,Marine,55.04,44.96
0,50,Manche,21.85,M. Emmanuel MACRON,176692,46.62,67.23,Mme Marine LE PEN,86133,22.72,32.77,résultats complets,Macron,Emmanuel,Le Pen,Marine,67.23,32.77
0,42,Loire,24.14,M. Emmanuel MACRON,218602,42.79,63.86,Mme Marine LE PEN,123713,24.21,36.14,résultats complets,Macron,Emmanuel,Le Pen,Marine,63.86,36.14
0,43,Haute-Loire,21.25,M. Emmanuel MACRON,76233,43.04,63.35,Mme Marine LE PEN,44110,24.90,36.65,résultats complets,Macron,Emmanuel,Le Pen,Marine,63.35,36.65
0,974,La Réunion,37.09,M. Emmanuel MACRON,212097,33.17,60.26,Mme Marine LE PEN,139873,21.88,39.74,résultats complets,Macron,Emmanuel,Le Pen,Marine,60.26,39.74
0,44,Loire-Atlantique,21.62,M. Emmanuel MACRON,530409,53.39,77.14,Mme Marine LE PEN,157184,15.82,22.86,résultats complets,Macron,Emmanuel,Le Pen,Marine,77.14,22.86
0,35,Ille-et-Vilaine,20.40,M. Emmanuel MACRON,403293,55.24,77.67,Mme Marine LE PEN,115942,15.88,22.33,résultats complets,Macron,Emmanuel,Le Pen,Marine,77.67,22.33
